In [1]:
if 0 :
    %matplotlib inline
else :
    %matplotlib notebook

#  Import  libraries

In [2]:
import sys
import os
module_path = os.path.abspath('.') +"\\_scripts"
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)
from _00_Import_packages_git3 import *

C:\Users\ng947ac\Documents\Daniel\jupyter\SoSTrades\jupyter_doc\_scripts
C:\Users\ng947ac\Documents\Daniel\git\workspace
C:\Users\ng947ac\Documents\Daniel\git\workspace\sosgemseo\src\
C:\Users\ng947ac\Documents\Daniel\git\workspace\sostrades-core


In [3]:
from time import sleep
from shutil import rmtree
from pathlib import Path
from os.path import join
import pandas as pd
import numpy as np
import os

from sos_trades_core.execution_engine.execution_engine import ExecutionEngine
from sos_trades_core.execution_engine.sos_simple_multi_scenario import SoSSimpleMultiScenario
from sos_trades_core.execution_engine.sos_very_simple_multi_scenario import SoSVerySimpleMultiScenario
from sos_trades_core.execution_engine.scatter_data import SoSScatterData
from sos_trades_core.execution_engine.sos_discipline_scatter import SoSDisciplineScatter
from tempfile import gettempdir
from sos_trades_core.tools.rw.load_dump_dm_data import DirectLoadDump
from sos_trades_core.study_manager.base_study_manager import BaseStudyManager
from sos_trades_core.execution_engine.sos_discipline import SoSDiscipline
from sos_trades_core.execution_engine.sos_coupling import SoSCoupling

# TestScatter

SoSDiscipline test class

# setUp

In [4]:
'''
Initialize third data needed for testing
'''
dirs_to_del = []
namespace = 'MyCase'
study_name = f'{namespace}'
repo = 'sos_trades_core.sos_processes.test'
base_path = 'sos_trades_core.sos_wrapping.test_discs'

root_dir = gettempdir()

In [5]:
exec_eng = ExecutionEngine(namespace)
factory = exec_eng.factory

# tearDown

In [6]:
for dir_to_del in dirs_to_del:
     sleep(0.5)
     if Path(dir_to_del).is_dir():
          rmtree(dir_to_del)
sleep(0.5)

# 02_consecutive_configuration 

In [7]:
exec_eng = ExecutionEngine(namespace)
factory = exec_eng.factory

In [8]:
# scatter build map
ac_map = {'input_name': 'name_list',
            'input_type': 'string_list',
            'input_ns': 'ns_scatter_scenario',
            'output_name': 'ac_name',
            'scatter_ns': 'ns_ac',
            'gather_ns': 'ns_scenario',
            'ns_to_update': ['ns_data_ac']}
exec_eng.smaps_manager.add_build_map('name_list', ac_map)
import pandas as pd
pd.DataFrame.from_dict(ac_map ,orient='index')

,0
input_name,name_list
input_type,string_list
input_ns,ns_scatter_scenario
output_name,ac_name
scatter_ns,ns_ac
gather_ns,ns_scenario
ns_to_update,[ns_data_ac]


In [9]:
# scenario build map
scenario_map =  {'input_name': 'scenario_list',
                        'input_type': 'string_list',
                        'input_ns': 'ns_scatter_scenario',
                        'output_name': 'scenario_name',
                        'scatter_ns': 'ns_scenario',
                        'gather_ns': 'ns_scatter_scenario',
                        'ns_to_update': ['ns_disc3', 'ns_barrierr', 'ns_out_disc3']}

exec_eng.smaps_manager.add_build_map(
     'scenario_list', scenario_map)
import pandas as pd
pd.DataFrame.from_dict(scenario_map ,orient='index')

,0
input_name,scenario_list
input_type,string_list
input_ns,ns_scatter_scenario
output_name,scenario_name
scatter_ns,ns_scenario
gather_ns,ns_scatter_scenario
ns_to_update,"[ns_disc3, ns_barrierr, ns_out_disc3]"


In [10]:
# shared namespace
exec_eng.ns_manager.add_ns('ns_barrierr', 'MyCase')
exec_eng.ns_manager.add_ns(
     'ns_scatter_scenario', 'MyCase.multi_scenarios')
exec_eng.ns_manager.add_ns(
     'ns_disc3', 'MyCase.multi_scenarios.Disc3')
exec_eng.ns_manager.add_ns(
     'ns_out_disc3', 'MyCase.multi_scenarios')
exec_eng.ns_manager.add_ns(
     'ns_data_ac', 'MyCase')

In [11]:
# instantiate factory # get instantiator from Discipline class

builder_list = factory.get_builder_from_process(repo=repo,
                                                                  mod_id='test_disc1_scenario')

scatter_list = exec_eng.factory.create_multi_scatter_builder_from_list(
     'name_list', builder_list=builder_list, autogather=True)

mod_path = f'{base_path}.disc3_scenario.Disc3'
disc3_builder = exec_eng.factory.get_builder_from_module(
     'Disc3', mod_path)
scatter_list.append(disc3_builder)

multi_scenarios = exec_eng.factory.create_very_simple_multi_scenario_builder(
     'multi_scenarios', 'scenario_list', scatter_list, autogather=True, gather_node='Post-processing')

exec_eng.factory.set_builders_to_coupling_builder(
     multi_scenarios)
exec_eng.configure()
exec_eng.display_treeview_nodes()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase is set to MDAJacobi
INFO:SoS.EE:Nodes representation for Treeview MyCase
|_ MyCase
	|_ multi_scenarios
	|_ Post-processing
		|_ Disc1
		|_ Disc3


'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3'

In [12]:
dict_values = {f'{study_name}.multi_scenarios.scenario_list': ['scenario_1', 'scenario_2'],
                  f'{study_name}.multi_scenarios.name_list': ['name_1', 'name_2']}
dict_values

{'MyCase.multi_scenarios.scenario_list': ['scenario_1', 'scenario_2'],
 'MyCase.multi_scenarios.name_list': ['name_1', 'name_2']}

In [13]:
exec_eng.load_study_from_input_dict(dict_values)
exec_eng.display_treeview_nodes()


INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.

'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t\t|_ scenario_1\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t\t|_ scenario_2\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3\n\t|_ name_1\n\t|_ name_2'

In [14]:
for disc in exec_eng.dm.get_disciplines_with_name('MyCase.multi_scenarios'):
     if isinstance(disc, SoSVerySimpleMultiScenario):
          print(list(disc.get_scattered_disciplines().keys()), [
                                    'scenario_1', 'scenario_2'])

['scenario_1', 'scenario_2'] ['scenario_1', 'scenario_2']


In [15]:
dict_values[study_name +
               '.multi_scenarios.scenario_list'] = ['scenario_1']
dict_values

{'MyCase.multi_scenarios.scenario_list': ['scenario_1'],
 'MyCase.multi_scenarios.name_list': ['name_1', 'name_2']}

In [16]:
exec_eng.load_study_from_input_dict(dict_values)
exec_eng.display_treeview_nodes()


INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase is set to MDAJacobi
ERROR:SoS.EE.DataManager:Variable: MyCase.multi_scenarios.scenario_1.Disc3.z value is not set!
ERROR:SoS.EE.DataManager:Variable: MyCase.multi_scenarios.scenario_1.Disc3.constant value is not set!
ERROR:SoS.EE.DataManager:Variable: MyCase.multi_scenarios.scenario_1.Disc3.power value is not set!
ERROR:SoS.EE.DataManager:Variable: MyCase.name_1.x value is not set!
ERROR:SoS.EE.DataManager:Variable: MyCase.name_1.a value is not set!
ERROR:SoS.EE.DataManager:Variable: MyCase.multi_scenarios.scenario_1.Disc1.name_1.b value is not set!
ERROR:SoS.EE.DataManager:Variable: MyCase.name_2.x value is not set!
ERROR:SoS.EE.DataManager:Variable: MyCase.name_2.a value is not set!
ERROR:SoS.EE.DataManager:Variable: MyCase.multi_scenarios.scenario_1.Disc1.name_2.b value is not set!
INFO:SoS.EE:Nodes representation for Treeview MyCase
|_ MyCase
	|_ multi_scenarios
		|_ s

'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t\t|_ scenario_1\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3\n\t|_ name_1\n\t|_ name_2'

In [17]:
print(
     [key for key in exec_eng.dm.data_id_map.keys()
      if 'scenario_2' in key and key.split('.')[-1] not in SoSDiscipline.NUM_DESC_IN and
      key.split('.')[-1] not in SoSCoupling.DEFAULT_NUMERICAL_PARAM_OUT_OF_INIT and key.split('.')[-1] != SoSCoupling.RESIDUALS_HISTORY],
     [])

[] []


In [18]:
for disc in exec_eng.dm.get_disciplines_with_name('MyCase.multi_scenarios'):
     if isinstance(disc, SoSVerySimpleMultiScenario):
          print(list(disc.get_scattered_disciplines().keys()), [
                                    'scenario_1'])

['scenario_1'] ['scenario_1']


In [19]:
dict_values[study_name +
               '.multi_scenarios.scenario_list'] = ['scenario_1', 'scenario_2', 'scenario_3']
dict_values

{'MyCase.multi_scenarios.scenario_list': ['scenario_1',
  'scenario_2',
  'scenario_3'],
 'MyCase.multi_scenarios.name_list': ['name_1', 'name_2']}

In [20]:
exec_eng.load_study_from_input_dict(dict_values)
exec_eng.display_treeview_nodes()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_3 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_3 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_1 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_2 is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_3 is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of t

'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t\t|_ scenario_1\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t\t|_ scenario_2\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t\t|_ scenario_3\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3\n\t|_ name_1\n\t|_ name_2'

In [21]:
for disc in exec_eng.dm.get_disciplines_with_name('MyCase.multi_scenarios'):
     if isinstance(disc, SoSVerySimpleMultiScenario):
          print(list(disc.get_scattered_disciplines().keys()), [
                                    'scenario_1', 'scenario_2', 'scenario_3'])

['scenario_1', 'scenario_2', 'scenario_3'] ['scenario_1', 'scenario_2', 'scenario_3']


In [22]:
dict_values[study_name +
               '.multi_scenarios.scenario_list'] = []
dict_values

{'MyCase.multi_scenarios.scenario_list': [],
 'MyCase.multi_scenarios.name_list': ['name_1', 'name_2']}

In [23]:
exec_eng.load_study_from_input_dict(dict_values)
exec_eng.display_treeview_nodes()

INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase is set to MDAJacobi
INFO:SoS.EE:Nodes representation for Treeview MyCase
|_ MyCase
	|_ multi_scenarios
	|_ Post-processing
		|_ Disc1
		|_ Disc3


'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3'

In [24]:
for disc in exec_eng.dm.get_disciplines_with_name('MyCase.multi_scenarios'):
     if isinstance(disc, SoSVerySimpleMultiScenario):
          print(
               list(disc.get_scattered_disciplines().keys()), [])

[] []


In [25]:
dict_values[study_name +
               '.multi_scenarios.scenario_list'] = ['scenario_A', 'scenario_B']
dict_values

{'MyCase.multi_scenarios.scenario_list': ['scenario_A', 'scenario_B'],
 'MyCase.multi_scenarios.name_list': ['name_1', 'name_2']}

In [26]:
print(
     [key for key in exec_eng.dm.data_id_map.keys() if 'scenario_1' in key and key.split('.')[-1] not in SoSDiscipline.NUM_DESC_IN and
      key.split('.')[-1] not in SoSCoupling.DEFAULT_NUMERICAL_PARAM_OUT_OF_INIT and key.split('.')[-1] != SoSCoupling.RESIDUALS_HISTORY], [])
print(
     [key for key in exec_eng.dm.data_id_map.keys() if 'scenario_2' in key and key.split('.')[-1] not in SoSDiscipline.NUM_DESC_IN and
      key.split('.')[-1] not in SoSCoupling.DEFAULT_NUMERICAL_PARAM_OUT_OF_INIT and key.split('.')[-1] != SoSCoupling.RESIDUALS_HISTORY], [])
print(
     [key for key in exec_eng.dm.data_id_map.keys() if 'scenario_3' in key and key.split('.')[-1] not in SoSDiscipline.NUM_DESC_IN and
      key.split('.')[-1] not in SoSCoupling.DEFAULT_NUMERICAL_PARAM_OUT_OF_INIT and key.split('.')[-1] != SoSCoupling.RESIDUALS_HISTORY], [])

[] []
[] []
[] []


In [27]:
exec_eng.load_study_from_input_dict(dict_values)
exec_eng.display_treeview_nodes()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_A is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_B is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_A is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_B is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_A is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_B is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_A is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase.multi_scenarios.scenario_B is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.

'Nodes representation for Treeview MyCase\n|_ MyCase\n\t|_ multi_scenarios\n\t\t|_ scenario_A\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t\t|_ scenario_B\n\t\t\t|_ Disc1\n\t\t\t\t|_ name_1\n\t\t\t\t|_ name_2\n\t\t\t|_ Disc3\n\t|_ Post-processing\n\t\t|_ Disc1\n\t\t|_ Disc3\n\t|_ name_1\n\t|_ name_2'

In [28]:
for disc in exec_eng.dm.get_disciplines_with_name('MyCase.multi_scenarios'):
     if isinstance(disc, SoSVerySimpleMultiScenario):
          print(list(disc.get_scattered_disciplines().keys()), [
                                    'scenario_A', 'scenario_B'])

['scenario_A', 'scenario_B'] ['scenario_A', 'scenario_B']


In [29]:
scenario_list = ['scenario_A', 'scenario_B']
for scenario in scenario_list:
     a1 = 3
     b1 = 4
     a2 = 6
     b2 = 2
     x1 = 2
     x2 = 4

     dict_values[study_name + '.name_1.a'] = a1
     dict_values[study_name + '.name_2.a'] = a2
     dict_values[study_name + '.multi_scenarios.' +
                    scenario + '.Disc1.name_1.b'] = b1
     dict_values[study_name + '.multi_scenarios.' +
                    scenario + '.Disc1.name_2.b'] = b2
     dict_values[study_name + '.multi_scenarios.' +
                    scenario + '.Disc3.constant'] = 3
     dict_values[study_name + '.multi_scenarios.' +
                    scenario + '.Disc3.power'] = 2
dict_values[study_name +
               '.multi_scenarios.scenario_A.Disc3.z'] = 1.2
dict_values[study_name +
               '.multi_scenarios.scenario_B.Disc3.z'] = 1.5
dict_values[study_name + '.name_1.x'] = x1
dict_values[study_name + '.name_2.x'] = x2
dict_values

{'MyCase.multi_scenarios.scenario_list': ['scenario_A', 'scenario_B'],
 'MyCase.multi_scenarios.name_list': ['name_1', 'name_2'],
 'MyCase.name_1.a': 3,
 'MyCase.name_2.a': 6,
 'MyCase.multi_scenarios.scenario_A.Disc1.name_1.b': 4,
 'MyCase.multi_scenarios.scenario_A.Disc1.name_2.b': 2,
 'MyCase.multi_scenarios.scenario_A.Disc3.constant': 3,
 'MyCase.multi_scenarios.scenario_A.Disc3.power': 2,
 'MyCase.multi_scenarios.scenario_B.Disc1.name_1.b': 4,
 'MyCase.multi_scenarios.scenario_B.Disc1.name_2.b': 2,
 'MyCase.multi_scenarios.scenario_B.Disc3.constant': 3,
 'MyCase.multi_scenarios.scenario_B.Disc3.power': 2,
 'MyCase.multi_scenarios.scenario_A.Disc3.z': 1.2,
 'MyCase.multi_scenarios.scenario_B.Disc3.z': 1.5,
 'MyCase.name_1.x': 2,
 'MyCase.name_2.x': 4}

In [30]:
exec_eng.load_study_from_input_dict(dict_values)

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling MyCase is set to MDAJacobi


In [31]:
exec_eng.execute()

INFO:SoS.EE:PROCESS EXECUTION MyCase STARTS...
INFO:SoS.EE:PROCESS EXECUTION MyCase ENDS.


{'MyCase.name_1.x': array([2]),
 'MyCase.name_1.a': array([3]),
 'MyCase.multi_scenarios.scenario_A.Disc1.name_1.b': array([4]),
 'MyCase.name_2.x': array([4]),
 'MyCase.name_2.a': array([6]),
 'MyCase.multi_scenarios.scenario_A.Disc1.name_2.b': array([2]),
 'MyCase.multi_scenarios.name_list': [1, 1],
 'MyCase.multi_scenarios.scenario_A.Disc3.z': array([1.2]),
 'MyCase.multi_scenarios.scenario_A.Disc3.constant': array([3]),
 'MyCase.multi_scenarios.scenario_A.Disc3.power': array([2]),
 'MyCase.multi_scenarios.scenario_B.Disc1.name_1.b': array([4]),
 'MyCase.multi_scenarios.scenario_B.Disc1.name_2.b': array([2]),
 'MyCase.multi_scenarios.scenario_B.Disc3.z': array([1.5]),
 'MyCase.multi_scenarios.scenario_B.Disc3.constant': array([3]),
 'MyCase.multi_scenarios.scenario_B.Disc3.power': array([2]),
 'MyCase.multi_scenarios.scenario_list': [1, 1],
 'MyCase.multi_scenarios.scenario_A.Disc1.name_1.indicator': array([12]),
 'MyCase.multi_scenarios.scenario_A.name_1.y': array([10]),
 'MyCase.m

In [32]:
for disc in exec_eng.dm.get_disciplines_with_name('MyCase.multi_scenarios'):
     if isinstance(disc, SoSVerySimpleMultiScenario):
          print(
               [key for key in list(disc.get_data_io_dict('in').keys()) if key not in disc.NUM_DESC_IN], ['scenario_list'])
          print(exec_eng.dm.get_value(
               f'{study_name}.multi_scenarios.scenario_list'), ['scenario_A', 'scenario_B'])
          print(
               list(exec_eng.dm.get_disciplines_with_name(
                    f'{study_name}')[0].get_sosdisc_outputs().keys()),
               ['residuals_history'])

     elif isinstance(disc, SoSScatterData):
          print(
               list(disc.get_data_io_dict('in').keys()), ['x_dict', 'scenario_list'])
          print(
               list(disc.get_data_io_dict('out').keys()), ['scenario_A.x', 'scenario_B.x'])
          print(exec_eng.dm.get_value(
               f'{study_name}.multi_scenarios.x_dict'), {'scenario_A': 2, 'scenario_B': 4})
          print(exec_eng.dm.get_value(
               f'{study_name}.multi_scenarios.scenario_A.x'), 2)
          print(exec_eng.dm.get_value(
               f'{study_name}.multi_scenarios.scenario_B.x'), 4)

['scenario_list'] ['scenario_list']
['scenario_A', 'scenario_B'] ['scenario_A', 'scenario_B']
['residuals_history'] ['residuals_history']
